----
# Visualizer
----

Use this notebook to: 
- visualize .stcs
- generate custom screenshots
- generate custom time series


In [27]:
import mne
from nilearn import plotting as nlplotting
import os
import glob
import pandas as pd
import numpy as np
from mayavi import mlab
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from configuration import subjects, subjects_dir, derivatives_root, n_jobs

os.environ['ETS_TOOLKIT'] = 'qt4'
os.environ['QT_API'] = 'pyqt5'
mne.viz.set_3d_backend('pyvista')

subsubj = "sub-" + str(subjects[0])

base_dir = os.path.join(derivatives_root, subsubj)
eventfolders = glob.glob(base_dir + "/spikes/*")
eventnames = list(map(lambda e: e.split("/")[-1], eventfolders))
modalities = ["eLORETA_with_peaks", "dSPM"]
eventdict = dict()

for ef in eventfolders:
    e = ef.split("/")[-1]
    stcs = glob.glob(ef +"/stc_*")
    mod_and_file = dict()
    for s in stcs:
        for m in modalities:
            if m in s:
                mod_and_file[m] = s
        eventdict[e] = mod_and_file

df = pd.DataFrame.from_dict(eventdict, orient='index')
df

,dSPM,eLORETA_with_peaks
Spike_Gr_1,/home/idrael/DATA/MEG/SourceLocTest/BIDSTestDa...,/home/idrael/DATA/MEG/SourceLocTest/BIDSTestDa...
Wickets,/home/idrael/DATA/MEG/SourceLocTest/BIDSTestDa...,/home/idrael/DATA/MEG/SourceLocTest/BIDSTestDa...


## Configuration
### Use the table from above, line-number = event-number (index starts at 0)

In [30]:
event_number    = 0      # Event (= line)
index           = 1      # Modality (= row)


#hemi = 'rh'           #right hemisphere
#hemi = 'lh'           #left hemisphere
hemi = 'split'        #both hemispheres in split view
#hemi = 'both'         #both hemispheres in anatomical view

views = ['lateral']
#views = ['medial']
#views = ['rostral']
#views = ['caudal']
#views = ['ventral']
#views = ['dorsal']
#views = ['lateral', 'medial']   # --> show lateral + medial simultaneously
#views = ['ventral', 'dorsal']
#views = ['lateral', 'medial', 'ventral', 'dorsal']

## Visualization
###    Don't touch from here, unless you know what you are doing :-)

In [31]:
stc = mne.read_source_estimate(df.iloc[event_number][index], subject=subsubj)

surfer_kwargs = dict(
    hemi=hemi, subjects_dir=subjects_dir,
    clim=dict(kind='percent', lims=[85, 97, 99]), 
    #clim=dict(kind='value', lims=[1e-12, 4.5e-12, 8e-12]),
    views=views,
    time_unit='ms', size=(800, 800),
    smoothing_steps=7)

brain = stc.plot(initial_time=0., **surfer_kwargs)
file_name = df.iloc[event_number][index]
file_name = file_name.split('-rh')[0]
file_name = file_name.split('-lh')[0]
text_to_add = str(file_name)
brain.add_text(0.1, 0.9, text_to_add, 'title', font_size=10)

Using control points [2.47499384e-11 3.92371449e-11 4.81302925e-11]


To do:

--> subsubj = "sub-" + str(subjects[0])
subjectname shoud be inferred from folder (10_visualizer.ipynb later will be moved to derivatives/subject) - no more need to unpack lists + works with multiple subjects...